In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch_geometric
import pickle
import optuna
from torch_geometric.nn import GATv2Conv

### data

In [2]:
# model structure
class GCN(torch.nn.Module):
    def __init__(self, size1, size2, size3):
        super(GCN, self).__init__()

        self.conv1 = GATv2Conv(1969, size1)
        self.conv2 = GATv2Conv(size1, size2)
        self.fc1 = torch.nn.Linear(size2, size3)
        self.fc2 = torch.nn.Linear(size3, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x1 = torch.relu(x)
        x = self.fc1(x1)
        x = torch.relu(x)
        x = self.fc2(x)       
        return x, x1

def train(model, data, optimizer, mask):
    model.train()
    optimizer.zero_grad()
    out, _ = model(data)
    loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out, _ = model(data)
        loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    return loss.item()

def objective(trial: optuna.Trial):   
    size1 = trial.suggest_int("size1", 32, 1024)
    size2 = trial.suggest_int("size2", 4, 512)
    size3 = trial.suggest_int("size3", 4, 512)
    num_epochs = trial.suggest_int("num_epochs", 1, 200)
    lr = trial.suggest_float("lr", 1e-5, 1e-2)

    model = GCN(size1, size2, size3).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # find best hyperparameter by training on the training set and get loss on the validating set
    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, data, optimizer, data.train_mask)
        val_loss = evaluate(model, data, data.valid_mask)
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return val_loss

In [ ]:
for rs in range(10):
    
    # read data
    X_train = pd.read_csv("../../result/input_perturb_phyloP/%d/X_train_stratified" % rs, sep="\t", index_col=0).values
    X_valid = pd.read_csv("../../result/input_perturb_phyloP/%d/X_valid_stratified" % rs, sep="\t", index_col=0).values
    X_test = pd.read_csv("../../result/input_perturb_phyloP/%d/X_test_stratified" % rs, sep="\t", index_col=0).values
    Y_train = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_train_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_valid = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_valid_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_test_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test_gene = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_test_stratified" % rs, sep="\t", index_col=0).index

    # concat all subsets
    X = np.concatenate([X_train, X_valid, X_test])
    Y = np.concatenate([Y_train, Y_valid, Y_test])
    train_mask = np.concatenate([[True] * len(X_train), [False] * len(X_valid), [False] * len(X_test)])
    valid_mask = np.concatenate([[False] * len(X_train), [True] * len(X_valid), [False] * len(X_test)])
    test_mask = np.concatenate([[False] * len(X_train), [False] * len(X_valid), [True] * len(X_test)])
    
    # mask of whether a test node is in the DAGMA graph
    dag = pd.read_csv("../../result/network_perturb_phyloP/DAGMA_thresholdAdaptive.tsv", sep="\t", header=None)
    id2genes = pd.read_csv("../../result/network_perturb_phyloP/valid_genes", sep="\t").set_index("ID")['genes'].to_dict()
    dag[0] = dag[0].map(id2genes)
    dag[1] = dag[1].map(id2genes)
    dag_genes = list(set.union(set(dag[0]), set(dag[1])))
    test_inDAG = Y_test_gene.isin(dag_genes)

    for network_name in ["DAGMA_thresholdAdaptive", "STRING", "BIOGRID", "Combine", "ChIP_hTFtarget", "ChIP_TIP", "ChIP_TIP_K562", "CoExpr_ENCODE_K562_0.75", "CoExpr_GTEx_WholeBlood_0.75", "CoExpr_perturb_0.5", "Random_ER", "Random_SF", "NOTEARS_thresholdAdaptive"]:

        # read network
        edge_index = torch.tensor(pd.read_csv("../../result/network_perturb_phyloP/%s.tsv" % network_name, sep="\t", header=None).values.T)

        # convert to data object
        data = torch_geometric.data.Data(x=torch.tensor(X).float(), edge_index=edge_index, y=torch.tensor(Y).float())
        data.train_mask = torch.tensor(train_mask)
        data.valid_mask = torch.tensor(valid_mask)
        data.test_mask = torch.tensor(test_mask)
        data.test_net_mask = np.concatenate([[False] * len(Y_train), [False] * len(Y_valid), test_inDAG])

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data = data.to(device)


        # repeat the whole process multiple times for mean and std of the loss
        test_loss_reps = []
        test_net_loss_reps = []

        os.makedirs("../../result/model_perturb_phyloP_GAT/%s/%d/" % (network_name, rs), exist_ok=True)

        for rep in range(10):
            # tune hyperparameters
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=20, timeout=600)
            best_params = study.best_params

            # final model with best hyperparameters and use train and valid data to train
            final_model = GCN(best_params["size1"], best_params["size2"], best_params["size3"]).to(device)
            optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])

            # train final model
            for epoch in range(1, best_params["num_epochs"] + 1):
                train_loss = train(final_model, data, optimizer, data.train_mask|data.valid_mask)

            # prediction from final model
            final_model.eval()
            with torch.no_grad():
                out, X1 = final_model(data)
                # loss on the testing set
                loss1 = F.mse_loss(out[data.test_mask].view(-1), data.y[data.test_mask])
                test_loss_reps.append(loss1.item())
                # loss on the testing set, but only the nodes in the DAGMA
                loss2 = F.mse_loss(out[data.test_net_mask].view(-1), data.y[data.test_net_mask])
                test_net_loss_reps.append(loss2.item())

            # save model, hyperparameters and results
            prefix = "../../result/model_perturb_phyloP_GAT/%s/%d/model%d" % (network_name, rs, rep)
            torch.save(final_model.state_dict(), prefix+".model")
            with open(prefix+".para", "wb") as f:
                pickle.dump([best_params, X1.cpu().numpy(), out.cpu().numpy(), loss1.item(), loss2.item()], f)

[I 2023-12-01 15:27:44,507] A new study created in memory with name: no-name-6dfee7a6-cec5-485d-b3ff-18c818baca5a
[I 2023-12-01 15:27:46,575] Trial 0 finished with value: 0.15259873867034912 and parameters: {'size1': 318, 'size2': 213, 'size3': 190, 'num_epochs': 98, 'lr': 0.00947664980369818}. Best is trial 0 with value: 0.15259873867034912.
[I 2023-12-01 15:27:46,772] Trial 1 finished with value: 0.1574244350194931 and parameters: {'size1': 993, 'size2': 53, 'size3': 136, 'num_epochs': 20, 'lr': 0.003687794739020309}. Best is trial 0 with value: 0.15259873867034912.
[I 2023-12-01 15:27:46,914] Trial 2 finished with value: 0.18416082859039307 and parameters: {'size1': 965, 'size2': 438, 'size3': 91, 'num_epochs': 11, 'lr': 0.0038161629239532456}. Best is trial 0 with value: 0.15259873867034912.
[I 2023-12-01 15:27:48,012] Trial 3 finished with value: 0.15806083381175995 and parameters: {'size1': 1021, 'size2': 315, 'size3': 463, 'num_epochs': 128, 'lr': 0.0096922692140585}. Best is tr

[I 2023-12-01 15:28:10,302] Trial 5 pruned. 
[I 2023-12-01 15:28:10,349] Trial 6 pruned. 
[I 2023-12-01 15:28:10,383] Trial 7 pruned. 
[I 2023-12-01 15:28:10,414] Trial 8 pruned. 
[I 2023-12-01 15:28:10,450] Trial 9 pruned. 
[I 2023-12-01 15:28:10,514] Trial 10 pruned. 
[I 2023-12-01 15:28:10,568] Trial 11 pruned. 
[I 2023-12-01 15:28:11,402] Trial 12 finished with value: 0.1475846916437149 and parameters: {'size1': 991, 'size2': 404, 'size3': 147, 'num_epochs': 91, 'lr': 0.0007380343131306059}. Best is trial 0 with value: 0.14746779203414917.
[I 2023-12-01 15:28:11,452] Trial 13 pruned. 
[I 2023-12-01 15:28:11,490] Trial 14 pruned. 
[I 2023-12-01 15:28:11,541] Trial 15 pruned. 
[I 2023-12-01 15:28:11,603] Trial 16 pruned. 
[I 2023-12-01 15:28:11,645] Trial 17 pruned. 
[I 2023-12-01 15:28:11,692] Trial 18 pruned. 
[I 2023-12-01 15:28:11,732] Trial 19 pruned. 
[I 2023-12-01 15:28:12,542] A new study created in memory with name: no-name-cc406349-b36c-45cf-b283-fb3d58e8e63d
[I 2023-12-01 

[I 2023-12-01 15:28:33,473] A new study created in memory with name: no-name-003295c2-22d0-419c-b836-671ca2dba380
[I 2023-12-01 15:28:33,777] Trial 0 finished with value: 0.14705945551395416 and parameters: {'size1': 885, 'size2': 196, 'size3': 51, 'num_epochs': 33, 'lr': 0.005540519069462886}. Best is trial 0 with value: 0.14705945551395416.
[I 2023-12-01 15:28:33,896] Trial 1 finished with value: 0.17943260073661804 and parameters: {'size1': 606, 'size2': 494, 'size3': 384, 'num_epochs': 12, 'lr': 0.006851821746771438}. Best is trial 0 with value: 0.14705945551395416.
[I 2023-12-01 15:28:34,079] Trial 2 finished with value: 0.15878865122795105 and parameters: {'size1': 46, 'size2': 285, 'size3': 432, 'num_epochs': 31, 'lr': 0.004923404939558217}. Best is trial 0 with value: 0.14705945551395416.
[I 2023-12-01 15:28:34,662] Trial 3 finished with value: 0.1534048318862915 and parameters: {'size1': 931, 'size2': 308, 'size3': 232, 'num_epochs': 68, 'lr': 0.006823541921013493}. Best is tr

[I 2023-12-01 15:29:08,294] Trial 5 pruned. 
[I 2023-12-01 15:29:08,354] Trial 6 pruned. 
[I 2023-12-01 15:29:10,261] Trial 7 finished with value: 0.18945296108722687 and parameters: {'size1': 671, 'size2': 290, 'size3': 224, 'num_epochs': 53, 'lr': 0.0005541412059083107}. Best is trial 2 with value: 0.1611652672290802.
[I 2023-12-01 15:29:10,335] Trial 8 pruned. 
[I 2023-12-01 15:29:10,390] Trial 9 pruned. 
[I 2023-12-01 15:29:10,551] Trial 10 finished with value: 0.1529393494129181 and parameters: {'size1': 74, 'size2': 173, 'size3': 11, 'num_epochs': 13, 'lr': 0.001959947014042278}. Best is trial 10 with value: 0.1529393494129181.
[I 2023-12-01 15:29:10,605] Trial 11 pruned. 
[I 2023-12-01 15:29:10,660] Trial 12 finished with value: 0.2412009835243225 and parameters: {'size1': 53, 'size2': 186, 'size3': 113, 'num_epochs': 3, 'lr': 0.0015691349948865232}. Best is trial 10 with value: 0.1529393494129181.
[I 2023-12-01 15:29:10,738] Trial 13 pruned. 
[I 2023-12-01 15:29:10,843] Trial 1

[I 2023-12-01 15:30:26,197] Trial 4 finished with value: 0.17068609595298767 and parameters: {'size1': 431, 'size2': 316, 'size3': 76, 'num_epochs': 73, 'lr': 0.00976947058948039}. Best is trial 0 with value: 0.1568593978881836.
[I 2023-12-01 15:30:27,835] Trial 5 finished with value: 0.17792974412441254 and parameters: {'size1': 381, 'size2': 10, 'size3': 34, 'num_epochs': 98, 'lr': 0.002188121741174547}. Best is trial 0 with value: 0.1568593978881836.
[I 2023-12-01 15:30:27,900] Trial 6 pruned. 
[I 2023-12-01 15:30:27,990] Trial 7 pruned. 
[I 2023-12-01 15:30:28,108] Trial 8 pruned. 
[I 2023-12-01 15:30:28,185] Trial 9 pruned. 
[I 2023-12-01 15:30:28,550] Trial 10 pruned. 
[I 2023-12-01 15:30:28,649] Trial 11 pruned. 
[I 2023-12-01 15:30:28,761] Trial 12 pruned. 
[I 2023-12-01 15:30:29,106] Trial 13 pruned. 
[I 2023-12-01 15:30:29,199] Trial 14 pruned. 
[I 2023-12-01 15:30:29,349] Trial 15 pruned. 
[I 2023-12-01 15:30:30,160] Trial 16 finished with value: 0.1547810435295105 and param

...

[I 2023-12-01 19:19:21,137] Trial 5 finished with value: 0.18273396790027618 and parameters: {'size1': 96, 'size2': 481, 'size3': 78, 'num_epochs': 9, 'lr': 0.006647207037468833}. Best is trial 3 with value: 0.17809446156024933.
[I 2023-12-01 19:19:31,556] Trial 6 finished with value: 0.17276757955551147 and parameters: {'size1': 573, 'size2': 39, 'size3': 126, 'num_epochs': 126, 'lr': 0.00488906083646414}. Best is trial 6 with value: 0.17276757955551147.
[I 2023-12-01 19:19:31,738] Trial 7 pruned. 
[I 2023-12-01 19:19:31,852] Trial 8 pruned. 
[I 2023-12-01 19:19:31,987] Trial 9 pruned. 
[I 2023-12-01 19:19:41,872] Trial 10 finished with value: 0.18710863590240479 and parameters: {'size1': 978, 'size2': 23, 'size3': 5, 'num_epochs': 73, 'lr': 0.002704308982620879}. Best is trial 6 with value: 0.17276757955551147.
[I 2023-12-01 19:19:55,818] Trial 11 finished with value: 0.17358043789863586 and parameters: {'size1': 699, 'size2': 81, 'size3': 196, 'num_epochs': 133, 'lr': 0.003998273168

[I 2023-12-01 19:22:09,603] Trial 2 finished with value: 0.17550386488437653 and parameters: {'size1': 227, 'size2': 399, 'size3': 69, 'num_epochs': 41, 'lr': 0.0021043283970819345}. Best is trial 2 with value: 0.17550386488437653.
[I 2023-12-01 19:22:11,272] Trial 3 finished with value: 0.17853344976902008 and parameters: {'size1': 613, 'size2': 264, 'size3': 331, 'num_epochs': 146, 'lr': 0.0023288282196630497}. Best is trial 2 with value: 0.17550386488437653.
[I 2023-12-01 19:22:11,703] Trial 4 finished with value: 0.17653511464595795 and parameters: {'size1': 696, 'size2': 471, 'size3': 220, 'num_epochs': 28, 'lr': 0.0026850429896322237}. Best is trial 2 with value: 0.17550386488437653.
[I 2023-12-01 19:22:11,734] Trial 5 pruned. 
[I 2023-12-01 19:22:11,788] Trial 6 pruned. 
[I 2023-12-01 19:22:11,836] Trial 7 pruned. 
[I 2023-12-01 19:22:11,870] Trial 8 pruned. 
[I 2023-12-01 19:22:11,903] Trial 9 pruned. 
[I 2023-12-01 19:22:12,464] Trial 10 finished with value: 0.1726173907518386

[I 2023-12-01 19:22:40,698] Trial 17 pruned. 
[I 2023-12-01 19:22:40,761] Trial 18 pruned. 
[I 2023-12-01 19:22:40,820] Trial 19 pruned. 
[I 2023-12-01 19:22:42,761] A new study created in memory with name: no-name-5877f111-99a9-4932-8c03-257ff1dc3e25
[I 2023-12-01 19:22:44,976] Trial 0 finished with value: 0.17648686468601227 and parameters: {'size1': 400, 'size2': 511, 'size3': 416, 'num_epochs': 198, 'lr': 0.008903775663877898}. Best is trial 0 with value: 0.17648686468601227.
[I 2023-12-01 19:22:46,173] Trial 1 finished with value: 0.17967556416988373 and parameters: {'size1': 296, 'size2': 379, 'size3': 358, 'num_epochs': 134, 'lr': 0.005425903341971434}. Best is trial 0 with value: 0.17648686468601227.
[I 2023-12-01 19:22:46,465] Trial 2 finished with value: 0.1760999709367752 and parameters: {'size1': 69, 'size2': 37, 'size3': 436, 'num_epochs': 53, 'lr': 0.007677684699849184}. Best is trial 2 with value: 0.1760999709367752.
[I 2023-12-01 19:22:46,729] Trial 3 finished with valu

[I 2023-12-01 19:23:22,804] Trial 4 finished with value: 0.1725209355354309 and parameters: {'size1': 661, 'size2': 483, 'size3': 125, 'num_epochs': 10, 'lr': 0.0006268608084930126}. Best is trial 4 with value: 0.1725209355354309.
[I 2023-12-01 19:23:22,858] Trial 5 pruned. 
[I 2023-12-01 19:23:22,899] Trial 6 pruned. 
[I 2023-12-01 19:23:22,951] Trial 7 pruned. 
[I 2023-12-01 19:23:22,977] Trial 8 pruned. 
[I 2023-12-01 19:23:23,015] Trial 9 pruned. 
[I 2023-12-01 19:23:23,065] Trial 10 pruned. 
[I 2023-12-01 19:23:23,140] Trial 11 pruned. 
[I 2023-12-01 19:23:23,211] Trial 12 pruned. 
[I 2023-12-01 19:23:23,725] Trial 13 finished with value: 0.16636331379413605 and parameters: {'size1': 52, 'size2': 437, 'size3': 13, 'num_epochs': 66, 'lr': 0.0017477258726476767}. Best is trial 13 with value: 0.16636331379413605.
[I 2023-12-01 19:23:24,064] Trial 14 finished with value: 0.17989201843738556 and parameters: {'size1': 98, 'size2': 422, 'size3': 5, 'num_epochs': 43, 'lr': 0.0018862216903

[I 2023-12-01 19:24:17,429] Trial 11 pruned. 
[I 2023-12-01 19:24:17,544] Trial 12 pruned. 
[I 2023-12-01 19:24:17,783] Trial 13 pruned. 
[I 2023-12-01 19:24:17,883] Trial 14 pruned. 
[I 2023-12-01 19:24:18,095] Trial 15 pruned. 
[I 2023-12-01 19:24:18,170] Trial 16 pruned. 
[I 2023-12-01 19:24:18,234] Trial 17 pruned. 
[I 2023-12-01 19:24:18,326] Trial 18 pruned. 
[I 2023-12-01 19:24:19,299] Trial 19 finished with value: 0.17163029313087463 and parameters: {'size1': 37, 'size2': 66, 'size3': 340, 'num_epochs': 162, 'lr': 0.0020153334158321723}. Best is trial 19 with value: 0.17163029313087463.
[I 2023-12-01 19:24:19,927] A new study created in memory with name: no-name-d77ce0ff-5971-4ac2-998a-7633dce9aa92
[I 2023-12-01 19:24:20,319] Trial 0 finished with value: 0.17684043943881989 and parameters: {'size1': 196, 'size2': 36, 'size3': 107, 'num_epochs': 51, 'lr': 0.009759266287658289}. Best is trial 0 with value: 0.17684043943881989.
[I 2023-12-01 19:24:20,771] Trial 1 finished with val